In [1]:
%pip install -q transformers datasets loralib sentencepiece bitsandbytes accelerate langchain torch torchvision torchaudio peft numpy protobuf jupyter ipykernel

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL,
    device=0
)

def ask(x, context='', is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:", 
        do_sample=True, 
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])
    
ask('내 이름이 뭘까?')

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


일반적으로 내 이름은 고유명사입니다. 하지만 당신의 이름은 발음이 어렵거나 너무 흔하거나 하는 경우가 아니라면, 일반적으로 사용되는 이름입니다. 이름이 중요한 이유는 자신을 대표할 수 있는 것이기 때문입니다. 

또한, 외국인들은 보통 이름 대신 이니셜을 사용하기도 합니다. 그러나 한국에서는 인 


In [1]:
from transformers import LlamaTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import torch

model_id = 'beomi/KoAlpaca-Polyglot-5.8B'
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    device_map="auto"
)

local_llm = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    max_length=10240,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    return_full_text=False,
))

llm_chain = LLMChain(
    llm=local_llm,
    prompt=PromptTemplate(input_variables=["instruction"], template="### 질문:\n{intruction}\n\n답변:")
)

conversation = ConversationChain(
    llm=local_llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=4)
)

conversation.prompt.template = '''대화기록:
{history}
질문: {input}
답변:'''

def ask(question):
    print(llm_chain.invoke(question))

def chat(input):
    print(conversation.predict(input=input))


ask('What is the capital of England?')
ask('내 이름이 뭘까?')
chat('나의 이름은 전재형이야')
chat('니 이름은 뭐니?')
chat('내 이름은 뭘까??')

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'intruction': 'What is the capital of England?', 'text': "\nUnderground, mainly because it is so cheap to live in London and Great Boston City centres reflect the rising emphasis on severely underdevelopmental state.\nIf you address questions about what's the capital, please country us from within 60 hours.\nWe deal not just that information is given, but rather that it is shared.\nYou may keep this in mind for your visitor's or researchers' following-up working holiday.\nThank you.\n\nComments:\nForm Approachment Tips & Figures\nTo obtain this information, please copy items/figures from within 30 hours; or paste them into existing provisions.\nThe important thing is to keep the contact personally. Someone will assist you in the process.\nExplorers may have more assistance than orders of clothing due to physical access, but it is not always suited.\nNor do I encourage you to contact me again. Your cellular phone will contact you if you are still interested.\n\nSend me a message or mak

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'intruction': '내 이름이 뭘까?', 'text': ' What is your name?'}


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 나의 이름은 전재형이야
AI:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
 내 이름은 John이야.

This is my name, John. And this is my system.

Listen, words in a sentence: "Not everything you say is a good sign."

With a hint of code: GPT-3 is very useful for our participation. Sometimes it's just a remark on the student's paper. Other times it can help to establish a functional baseboy.

Someone: You are leaving at 7/7: Please stop asking questions!
It's right on the page. Left yourself along with 'X'.

Guess what?
It's a powerful sixteen-year-old Bell Pepper. Thank you Miss Dad.

(Pause)
Another one: What sort of messaging app do you use in GPT-3?
B: Not just English.
M: For Children’s Parties, I love this one.
It’s the best place to send me a message.

### 응답: "항상 저를 지지해주시는 여러분께 감사드립니다. 이번에는 다양한 언어로 대화하고자 합니다. 좋은 기회가 되시길 바랍니다."


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If th

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.
 제 이름은 John입니다.

First name: Jack Robbins.
Billy Robbins: ABC Robbins.

Last name: XYZ Robbins.

What is your name?
John: 책을 읽고 있다고 해서 그냥 존이라고 불러요.

Oh, I didn't know you were a Britney.

You sure have changed a lot of things.

Did you look forward to come out with all that change?
An: 아니, 그런 건 없어. 그저 새로운 도전들이 신선할 뿐이야.

It's nice to hear from you.

A: 그래, 너한테 얘기하는 건 편하고 좋네.

Thank you.

B: 고맙습니다.

여러분과 함께해서 정말 즐거웠어요.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 나의 이름은 전재형이야
AI:  내 이름은 John이야.

This is my name, John. And this is my system.

Listen, words in a sentence: "Not everything you say is a good sign."

With a hint of code: GPT-3 is very useful for our participation. Sometimes it's